<a target="_blank" href="https://colab.research.google.com/github/LuisAngelMendozaVelasco/Applied_Data_Science_with_Python_Specialization/blob/main/Introduction_to_Data_Science_in_Python/Week4/Labs/assignment4.ipynb"><img src="https://www.tensorflow.org/images/colab_logo_32px.png" />Run in Google Colab</a>

In [1]:
%%bash
# If running in Google Colab

# mkdir assets
# wget https://raw.githubusercontent.com/LuisAngelMendozaVelasco/Applied_Data_Science_with_Python_Specialization/main/Introduction_to_Data_Science_in_Python/Week4/Labs/assets/mlb.csv -P ./assets
# wget https://raw.githubusercontent.com/LuisAngelMendozaVelasco/Applied_Data_Science_with_Python_Specialization/main/Introduction_to_Data_Science_in_Python/Week4/Labs/assets/nba.csv -P ./assets
# wget https://raw.githubusercontent.com/LuisAngelMendozaVelasco/Applied_Data_Science_with_Python_Specialization/main/Introduction_to_Data_Science_in_Python/Week4/Labs/assets/nfl.csv -P ./assets
# wget https://raw.githubusercontent.com/LuisAngelMendozaVelasco/Applied_Data_Science_with_Python_Specialization/main/Introduction_to_Data_Science_in_Python/Week4/Labs/assets/nhl.csv -P ./assets
# wget https://raw.githubusercontent.com/LuisAngelMendozaVelasco/Applied_Data_Science_with_Python_Specialization/main/Introduction_to_Data_Science_in_Python/Week4/Labs/assets/wikipedia_data.html -P ./assets

# Assignment 4

## Description

In this assignment you must read in a file of metropolitan regions and associated sports teams from [assets/wikipedia_data.html](assets/wikipedia_data.html) and answer some questions about each metropolitan region. Each of these regions may have one or more teams from the "Big 4": NFL (football, in [assets/nfl.csv](assets/nfl.csv)), MLB (baseball, in [assets/mlb.csv](assets/mlb.csv)), NBA (basketball, in [assets/nba.csv](assets/nba.csv) or NHL (hockey, in [assets/nhl.csv](assets/nhl.csv)). Please keep in mind that all questions are from the perspective of the metropolitan region, and that this file is the "source of authority" for the location of a given sports team. Thus teams which are commonly known by a different area (e.g. "Oakland Raiders") need to be mapped into the metropolitan region given (e.g. San Francisco Bay Area). This will require some human data understanding outside of the data you've been given (e.g. you will have to hand-code some names, and might need to google to find out where teams are)!

For each sport I would like you to answer the question: **what is the win/loss ratio's correlation with the population of the city it is in?** Win/Loss ratio refers to the number of wins over the number of wins plus the number of losses. Remember that to calculate the correlation with [`pearsonr`](https://docs.scipy.org/doc/scipy/reference/generated/scipy.stats.pearsonr.html), so you are going to send in two ordered lists of values, the populations from the wikipedia_data.html file and the win/loss ratio for a given sport in the same order. Average the win/loss ratios for those cities which have multiple teams of a single sport. Each sport is worth an equal amount in this assignment (20%\*4=80%) of the grade for this assignment. You should only use data **from year 2018** for your analysis -- this is important!

## Notes

1. Do not include data about the MLS or CFL in any of the work you are doing, we're only interested in the Big 4 in this assignment.
2. I highly suggest that you first tackle the four correlation questions in order, as they are all similar and worth the majority of grades for this assignment. This is by design!
3. It's fair game to talk with peers about high level strategy as well as the relationship between metropolitan areas and sports teams. However, do not post code solving aspects of the assignment (including such as dictionaries mapping areas to teams, or regexes which will clean up names).
4. There may be more teams than the assert statements test, remember to collapse multiple teams in one city into a single value!

## Question 1

For this question, calculate the win/loss ratio's correlation with the population of the city it is in for the **NHL** using **2018** data.

In [2]:
import pandas as pd
import numpy as np
import scipy.stats as stats
import re

nhl_df = pd.read_csv("./assets/nhl.csv")
cities = pd.read_html("./assets/wikipedia_data.html")[1]
cities = cities.iloc[:-1, [0, 3, 5, 6, 7, 8]]

In [3]:
nhl_df.head()

,team,GP,W,L,OL,PTS,PTS%,GF,GA,SRS,SOS,RPt%,ROW,year,League
0,Atlantic Division,Atlantic Division,Atlantic Division,Atlantic Division,Atlantic Division,Atlantic Division,Atlantic Division,Atlantic Division,Atlantic Division,Atlantic Division,Atlantic Division,Atlantic Division,Atlantic Division,2018,NHL
1,Tampa Bay Lightning*,82,54,23,5,113,.689,296,236,0.66,-0.07,.634,48,2018,NHL
2,Boston Bruins*,82,50,20,12,112,.683,270,214,0.62,-0.07,.610,47,2018,NHL
3,Toronto Maple Leafs*,82,49,26,7,105,.640,277,232,0.49,-0.06,.567,42,2018,NHL
4,Florida Panthers,82,44,30,8,96,.585,248,246,-0.01,-0.04,.537,41,2018,NHL


In [4]:
nhl_df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 171 entries, 0 to 170
Data columns (total 15 columns):
 #   Column  Non-Null Count  Dtype 
---  ------  --------------  ----- 
 0   team    171 non-null    object
 1   GP      171 non-null    object
 2   W       171 non-null    object
 3   L       171 non-null    object
 4   OL      171 non-null    object
 5   PTS     171 non-null    object
 6   PTS%    171 non-null    object
 7   GF      171 non-null    object
 8   GA      171 non-null    object
 9   SRS     171 non-null    object
 10  SOS     171 non-null    object
 11  RPt%    171 non-null    object
 12  ROW     171 non-null    object
 13  year    171 non-null    int64 
 14  League  171 non-null    object
dtypes: int64(1), object(14)
memory usage: 20.2+ KB


In [5]:
cities.head()

,Metropolitan area,Population (2016 est.)[8],NFL,MLB,NBA,NHL
0,New York City,20153634,Giants Jets[note 1],Yankees Mets[note 2],Knicks Nets,Rangers Islanders Devils[note 3]
1,Los Angeles,13310447,Rams Chargers[note 4],Dodgers Angels,Lakers Clippers,Kings Ducks
2,San Francisco Bay Area,6657982,49ers Raiders[note 6],Giants Athletics,Warriors,Sharks[note 7]
3,Chicago,9512999,Bears[note 8],Cubs White Sox,Bulls[note 9],Blackhawks
4,Dallas–Fort Worth,7233323,Cowboys,Rangers,Mavericks,Stars


In [6]:
cities.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 51 entries, 0 to 50
Data columns (total 6 columns):
 #   Column                     Non-Null Count  Dtype 
---  ------                     --------------  ----- 
 0   Metropolitan area          51 non-null     object
 1   Population (2016 est.)[8]  51 non-null     object
 2   NFL                        51 non-null     object
 3   MLB                        51 non-null     object
 4   NBA                        51 non-null     object
 5   NHL                        51 non-null     object
dtypes: object(6)
memory usage: 2.5+ KB


In [7]:
def clean_nhl_df():
    # Load data
    nhl_df = pd.read_csv("./assets/nhl.csv")
    cities = pd.read_html("./assets/wikipedia_data.html")[1]
    cities = cities.iloc[:-1, [0, 3, 5, 6, 7, 8]]

    # Cleaning the cities dataframe
    cities["NHL"] = cities["NHL"].apply(lambda x: re.sub(r"\[.+\]", "", x))
    cities["NHL"] = cities["NHL"].replace({"Rangers Islanders Devils": "Rangers,Islanders,Devils",
                                           "Kings Ducks": "Kings,Ducks",
                                           "Red Wings": "Red,Wings", 
                                           "Maple Leafs": "Maple,Leafs", 
                                           "Blue Jackets": "Blue,Jackets",
                                           "Golden Knights": "Golden,Knights"})
    cities["NHL"] = cities["NHL"].apply(lambda x: x.split(","))
    cities = cities.explode("NHL")

    # Cleaning the nhl_df dataframe
    nhl_df = nhl_df[nhl_df["year"] == 2018]
    nhl_df["team"] = nhl_df["team"].apply(lambda x: x.replace("*", ""))
    nhl_df["team"] = nhl_df["team"].apply(lambda x: x.split(" ")[-1])

    # Merge the dataframes
    df = pd.merge(cities, nhl_df, left_on="NHL", right_on="team")
    df = df[["Metropolitan area", "Population (2016 est.)[8]", "NHL", "team", "W", "L"]]
    df["W-L%"] = df["W"].astype("int") / (df["W"].astype("int") + df["L"].astype("int"))

    # Drop duplicated rows
    df.loc[df["Metropolitan area"] == "New York City", "W-L%"] = df.loc[df["Metropolitan area"] == "New York City", "W-L%"].mean()
    df.loc[df["Metropolitan area"] == "Los Angeles", "W-L%"] = df.loc[df["Metropolitan area"] == "Los Angeles", "W-L%"].mean()
    df = df.drop_duplicates(subset="Metropolitan area").reset_index().drop(columns="index")
    
    return df

def nhl_correlation(): 
    # YOUR CODE HERE
    df = clean_nhl_df()
    population_by_region = df["Population (2016 est.)[8]"].astype("int") # pass in metropolitan area population from cities
    win_loss_by_region = df["W-L%"].astype("float") # pass in win/loss ratio from nhl_df in the same order as cities["Metropolitan area"]

    assert len(population_by_region) == len(win_loss_by_region), "Q1: Your lists must be the same length"
    assert len(population_by_region) == 28, "Q1: There should be 28 teams being analysed for NHL"
    
    return stats.pearsonr(population_by_region, win_loss_by_region)[0]

    # raise NotImplementedError()

In [8]:
nhl_correlation()

0.012486162921209909

## Question 2

For this question, calculate the win/loss ratio's correlation with the population of the city it is in for the **NBA** using **2018** data.

In [9]:
import pandas as pd
import numpy as np
import scipy.stats as stats
import re

nba_df = pd.read_csv("./assets/nba.csv")
cities = pd.read_html("./assets/wikipedia_data.html")[1]
cities = cities.iloc[:-1, [0, 3, 5, 6, 7, 8]]

In [10]:
nba_df.head()

,team,W,L,W/L%,GB,PS/G,PA/G,SRS,year,League
0,Toronto Raptors* (1),59,23,0.72,—,111.7,103.9,7.29,2018,NBA
1,Boston Celtics* (2),55,27,0.6709999999999999,4.0,104.0,100.4,3.23,2018,NBA
2,Philadelphia 76ers* (3),52,30,0.634,7.0,109.8,105.3,4.3,2018,NBA
3,Cleveland Cavaliers* (4),50,32,0.61,9.0,110.9,109.9,0.59,2018,NBA
4,Indiana Pacers* (5),48,34,0.585,11.0,105.6,104.2,1.18,2018,NBA


In [11]:
nba_df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 162 entries, 0 to 161
Data columns (total 10 columns):
 #   Column  Non-Null Count  Dtype 
---  ------  --------------  ----- 
 0   team    162 non-null    object
 1   W       162 non-null    object
 2   L       162 non-null    object
 3   W/L%    162 non-null    object
 4   GB      162 non-null    object
 5   PS/G    162 non-null    object
 6   PA/G    162 non-null    object
 7   SRS     162 non-null    object
 8   year    162 non-null    int64 
 9   League  162 non-null    object
dtypes: int64(1), object(9)
memory usage: 12.8+ KB


In [12]:
def clean_nba_df():
    # Load data
    nba_df = pd.read_csv("./assets/nba.csv")
    cities = pd.read_html("./assets/wikipedia_data.html")[1]
    cities = cities.iloc[:-1, [0, 3, 5, 6, 7, 8]]
    
    # Cleaning the cities dataframe
    cities["NBA"] = cities["NBA"].apply(lambda x: re.sub(r"\[.+\]", "", x))
    cities["NBA"] = cities["NBA"].replace({"Knicks Nets": "Knicks,Nets",
                                           "Lakers Clippers": "Lakers,Clippers",
                                           "Trail Blazers": "Trail,Blazers"})
    cities["NBA"] = cities["NBA"].apply(lambda x: x.split(","))
    cities = cities.explode("NBA")

    # Cleaning the nba_df dataframe
    nba_df = nba_df[nba_df["year"] == 2018]
    nba_df["team"] = nba_df["team"].apply(lambda x: re.sub(r"(\*)*\s\(\d+\)", "", x))
    nba_df["team"] = nba_df["team"].apply(lambda x: x.split(" ")[-1])

    # Merge the dataframes
    df = pd.merge(cities, nba_df, left_on="NBA", right_on="team")
    df = df.rename(columns={"W/L%": "W-L%"})
    df = df[["Metropolitan area", "Population (2016 est.)[8]", "NBA", "team", "W", "L", "W-L%"]]
    df["W-L%"] = df["W-L%"].astype("float")
    
    # Drop duplicated columns
    df.loc[df["Metropolitan area"] == "New York City", "W-L%"] = df.loc[df["Metropolitan area"] == "New York City", "W-L%"].mean()
    df.loc[df["Metropolitan area"] == "Los Angeles", "W-L%"] = df.loc[df["Metropolitan area"] == "Los Angeles", "W-L%"].mean()
    df = df.drop_duplicates(subset="Metropolitan area").reset_index().drop(columns="index")
    
    return df

def nba_correlation():
    # YOUR CODE HERE
    df = clean_nba_df()
    population_by_region = df["Population (2016 est.)[8]"].astype("int") # pass in metropolitan area population from cities
    win_loss_by_region = df["W-L%"] # pass in win/loss ratio from nba_df in the same order as cities["Metropolitan area"]

    assert len(population_by_region) == len(win_loss_by_region), "Q2: Your lists must be the same length"
    assert len(population_by_region) == 28, "Q2: There should be 28 teams being analysed for NBA"

    return stats.pearsonr(population_by_region, win_loss_by_region)[0]

    # raise NotImplementedError()

In [13]:
nba_correlation()

-0.17636350642182935

## Question 3

For this question, calculate the win/loss ratio's correlation with the population of the city it is in for the **MLB** using **2018** data.

In [14]:
import pandas as pd
import numpy as np
import scipy.stats as stats
import re

mlb_df = pd.read_csv("./assets/mlb.csv")
cities = pd.read_html("./assets/wikipedia_data.html")[1]
cities = cities.iloc[:-1, [0, 3, 5, 6, 7, 8]]

In [15]:
mlb_df.head()

,team,W,L,W-L%,GB,year,League
0,Boston Red Sox,108,54,0.667,--,2018,MLB
1,New York Yankees,100,62,0.617,8.0,2018,MLB
2,Tampa Bay Rays,90,72,0.556,18.0,2018,MLB
3,Toronto Blue Jays,73,89,0.451,35.0,2018,MLB
4,Baltimore Orioles,47,115,0.290,61.0,2018,MLB


In [16]:
mlb_df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 150 entries, 0 to 149
Data columns (total 7 columns):
 #   Column  Non-Null Count  Dtype  
---  ------  --------------  -----  
 0   team    150 non-null    object 
 1   W       150 non-null    int64  
 2   L       150 non-null    int64  
 3   W-L%    150 non-null    float64
 4   GB      150 non-null    object 
 5   year    150 non-null    int64  
 6   League  150 non-null    object 
dtypes: float64(1), int64(3), object(3)
memory usage: 8.3+ KB


In [17]:
def clean_mlb_df():
    # Load data
    mlb_df = pd.read_csv("./assets/mlb.csv")
    cities = pd.read_html("./assets/wikipedia_data.html")[1]
    cities = cities.iloc[:-1, [0, 3, 5, 6, 7, 8]]

    # Cleaning the cities dataframe
    cities["MLB"] = cities["MLB"].apply(lambda x: re.sub(r"\[.+\]", "", x))
    cities["MLB"] = cities["MLB"].replace({"Blue Jays": "Blue,Jays", 
                                           "Cubs White Sox": "Cubs,WhiteSox", 
                                           "Dodgers Angels": "Dodgers,Angels", 
                                           "Giants Athletics": "Giants,Athletics", 
                                           "Yankees Mets": "Yankees,Mets",
                                           "Red Sox": "RedSox"})
    cities["MLB"] = cities["MLB"].apply(lambda x: x.split(","))
    cities = cities.explode("MLB")

    # Cleaning the mlb_df dataframe
    mlb_df = mlb_df[mlb_df["year"] == 2018]
    mlb_df["team"] = mlb_df["team"].replace({"Boston Red Sox": "Boston RedSox",
                                             "Chicago White Sox": "Chicago WhiteSox"})
    mlb_df["team"] = mlb_df["team"].apply(lambda x: x.split(" ")[-1])

    # Merge the dataframes
    df = pd.merge(cities, mlb_df, left_on="MLB", right_on="team")
    df = df[["Metropolitan area", "Population (2016 est.)[8]", "MLB", "team", "W", "L", "W-L%"]]
    df["W-L%"] = df["W-L%"].astype("float")

    # Drop duplicated columns
    df.loc[df["Metropolitan area"] == "New York City", "W-L%"] = df.loc[df["Metropolitan area"] == "New York City", "W-L%"].mean()
    df.loc[df["Metropolitan area"] == "Los Angeles", "W-L%"] = df.loc[df["Metropolitan area"] == "Los Angeles", "W-L%"].mean()
    df.loc[df["Metropolitan area"] == "San Francisco Bay Area", "W-L%"] = df.loc[df["Metropolitan area"] == "San Francisco Bay Area", "W-L%"].mean()
    df.loc[df["Metropolitan area"] == "Chicago", "W-L%"] = df.loc[df["Metropolitan area"] == "Chicago", "W-L%"].mean()
    df = df.drop_duplicates(subset="Metropolitan area").reset_index().drop(columns="index")
    
    return df

def mlb_correlation(): 
    # YOUR CODE HERE
    df = clean_mlb_df()
    population_by_region = df["Population (2016 est.)[8]"].astype("int") # pass in metropolitan area population from cities
    win_loss_by_region = df["W-L%"] # pass in win/loss ratio from mlb_df in the same order as cities["Metropolitan area"]

    assert len(population_by_region) == len(win_loss_by_region), "Q3: Your lists must be the same length"
    assert len(population_by_region) == 26, "Q3: There should be 26 teams being analysed for MLB"

    return stats.pearsonr(population_by_region, win_loss_by_region)[0]

    # raise NotImplementedError()

In [18]:
mlb_correlation()

0.15003737475409493

## Question 4

For this question, calculate the win/loss ratio's correlation with the population of the city it is in for the **NFL** using **2018** data.

In [19]:
import pandas as pd
import numpy as np
import scipy.stats as stats
import re

nfl_df = pd.read_csv("./assets/nfl.csv")
cities = pd.read_html("./assets/wikipedia_data.html")[1]
cities = cities.iloc[:-1, [0, 3, 5, 6, 7, 8]]

In [20]:
nfl_df.head()

,DSRS,L,League,MoV,OSRS,PA,PD,PF,SRS,SoS,T,W,W-L%,team,year
0,AFC East,AFC East,NFL,AFC East,AFC East,AFC East,AFC East,AFC East,AFC East,AFC East,AFC East,AFC East,AFC East,AFC East,2018
1,2.1,5,NFL,6.9,3.1,325,111,436,5.2,-1.8,0,11,.688,New England Patriots*,2018
2,-5.2,9,NFL,-7.1,-3.6,433,-114,319,-8.8,-1.7,0,7,.438,Miami Dolphins,2018
3,-0.6,10,NFL,-6.6,-6.3,374,-105,269,-6.9,-0.3,0,6,.375,Buffalo Bills,2018
4,-5.9,12,NFL,-6.8,-2.0,441,-108,333,-7.8,-1.1,0,4,.250,New York Jets,2018


In [21]:
nfl_df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 200 entries, 0 to 199
Data columns (total 15 columns):
 #   Column  Non-Null Count  Dtype 
---  ------  --------------  ----- 
 0   DSRS    200 non-null    object
 1   L       200 non-null    object
 2   League  200 non-null    object
 3   MoV     200 non-null    object
 4   OSRS    200 non-null    object
 5   PA      200 non-null    object
 6   PD      200 non-null    object
 7   PF      200 non-null    object
 8   SRS     200 non-null    object
 9   SoS     200 non-null    object
 10  T       120 non-null    object
 11  W       200 non-null    object
 12  W-L%    200 non-null    object
 13  team    200 non-null    object
 14  year    200 non-null    int64 
dtypes: int64(1), object(14)
memory usage: 23.6+ KB


In [22]:
def clean_nfl_df():
    # Load data
    nfl_df = pd.read_csv("./assets/nfl.csv")
    cities = pd.read_html("./assets/wikipedia_data.html")[1]
    cities = cities.iloc[:-1, [0, 3, 5, 6, 7, 8]]

    # Cleaning the cities dataframe
    cities["NFL"] = cities["NFL"].apply(lambda x: re.sub(r"\[.+\]", "", x))
    cities["NFL"] = cities["NFL"].replace({"Giants Jets": "Giants,Jets",
                                           "Rams Chargers": "Rams,Chargers",
                                           "49ers Raiders": "49ers,Raiders"})
    cities["NFL"] = cities["NFL"].apply(lambda x: x.split(","))
    cities = cities.explode("NFL")

    # Cleaning the nfl_df dataframe
    nfl_df = nfl_df[nfl_df["year"] == 2018]
    nfl_df["team"] = nfl_df["team"].apply(lambda x: re.sub(r"(\*|\+)", "", x))
    nfl_df["team"] = nfl_df["team"].apply(lambda x: x.split(" ")[-1])

    # Merge the dataframes
    df = pd.merge(cities, nfl_df, left_on="NFL", right_on="team")
    df = df[["Metropolitan area", "Population (2016 est.)[8]", "NFL", "team", "W", "L", "W-L%"]]
    df["W-L%"] = df["W-L%"].astype("float")

    # drop duplicated columns
    df.loc[df["Metropolitan area"] == "New York City", "W-L%"] = df.loc[df["Metropolitan area"] == "New York City", "W-L%"].mean()
    df.loc[df["Metropolitan area"] == "Los Angeles", "W-L%"] = df.loc[df["Metropolitan area"] == "Los Angeles", "W-L%"].mean()
    df.loc[df["Metropolitan area"] == "San Francisco Bay Area", "W-L%"] = df.loc[df["Metropolitan area"] == "San Francisco Bay Area", "W-L%"].mean()
    df = df.drop_duplicates(subset="Metropolitan area").reset_index().drop(columns="index")
    
    return df

def nfl_correlation(): 
    # YOUR CODE HERE
    df = clean_nfl_df()
    population_by_region = df["Population (2016 est.)[8]"].astype("int") # pass in metropolitan area population from cities
    win_loss_by_region = df["W-L%"] # pass in win/loss ratio from nfl_df in the same order as cities["Metropolitan area"]

    assert len(population_by_region) == len(win_loss_by_region), "Q4: Your lists must be the same length"
    assert len(population_by_region) == 29, "Q4: There should be 29 teams being analysed for NFL"

    return stats.pearsonr(population_by_region, win_loss_by_region)[0]

    # raise NotImplementedError()

In [23]:
nfl_correlation()

0.004282141436393008

## Question 5

In this question I would like you to explore the hypothesis that **given that an area has two sports teams in different sports, those teams will perform the same within their respective sports**. How I would like to see this explored is with a series of paired t-tests (so use [`ttest_rel`](https://docs.scipy.org/doc/scipy/reference/generated/scipy.stats.ttest_rel.html)) between all pairs of sports. Are there any sports where we can reject the null hypothesis? Again, average values where a sport has multiple teams in one region. Remember, you will only be including, for each sport, cities which have teams engaged in that sport, drop others as appropriate. This question is worth 20% of the grade for this assignment.

In [24]:
import pandas as pd
import numpy as np
import scipy.stats as stats
import re

# mlb_df = pd.read_csv("assets/mlb.csv")
# nhl_df = pd.read_csv("assets/nhl.csv")
# nba_df = pd.read_csv("assets/nba.csv")
# nfl_df = pd.read_csv("assets/nfl.csv")
# cities = pd.read_html("assets/wikipedia_data.html")[1]
# cities = cities.iloc[:-1,[0,3,5,6,7,8]]

def clean_dfs():
    nfl_df = clean_nfl_df()
    nba_df = clean_nba_df()
    nhl_df = clean_nhl_df()   
    mlb_df = clean_mlb_df()
    
    nfl_df = nfl_df[["Metropolitan area", "W-L%"]]
    nba_df = nba_df[["Metropolitan area", "W-L%"]]
    nhl_df = nhl_df[["Metropolitan area", "W-L%"]]  
    mlb_df = mlb_df[["Metropolitan area", "W-L%"]]
    
    return nfl_df, nba_df, nhl_df, mlb_df

def calculate_p_values(leagues):
    p_values = pd.DataFrame(columns=leagues.keys(), index=leagues.keys())
    for (league_name1, league_df1) in leagues.items():
        for (league_name2, league_df2) in leagues.items():
            merged_league = pd.merge(league_df1, league_df2, on="Metropolitan area")
            p_values.loc[league_name1, league_name2] = stats.ttest_rel(merged_league["W-L%_x"], merged_league["W-L%_y"])[1]
            
    return p_values

def sports_team_performance():
    # YOUR CODE HERE
    # Note: p_values is a full dataframe, so df.loc["NFL","NBA"] should be the same as df.loc["NBA","NFL"] and df.loc["NFL","NFL"] should return np.nan
    # sports = ['NFL', 'NBA', 'NHL', 'MLB']
    # p_values = pd.DataFrame({k:np.nan for k in sports}, index=sports)
    nfl_df, nba_df, nhl_df, mlb_df = clean_dfs()
    leagues = {"NFL": nfl_df, "NBA": nba_df, "NHL": nhl_df, "MLB": mlb_df}
    p_values = calculate_p_values(leagues)   
    
    assert abs(p_values.loc["NBA", "NHL"] - 0.02) <= 1e-2, "The NBA-NFL p-value should be around 0.02"
    assert abs(p_values.loc["MLB", "NFL"] - 0.80) <= 1e-2, "The MLB-NFL p-value should be around 0.80"
    
    return p_values

    # raise NotImplementedError()

In [25]:
sports_team_performance()

,NFL,NBA,NHL,MLB
NFL,NaN,0.937509,0.030318,0.803459
NBA,0.937509,NaN,0.022386,0.949566
NHL,0.030318,0.022386,NaN,0.000703
MLB,0.803459,0.949566,0.000703,NaN
